In [1]:
!python3 cifar_train_lorot-E.py --exp_str lorote --workers 8 --epochs 200 --gpu 0 

cifar_train_lorot-E.py:167: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn(
Use GPU: 0 for training
=> creating model 'resnet32'
num_trans : 16
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]
Epoch: [0][0/97], lr: 0.00200	Time 4.160 (4.160)	Data 0.538 (0.538)	Loss 6.2761 (6.2761)	Prec@1 2.344 (2.344)	Prec@5 35.156 (35.156)
Epoch: [0][10/97], lr: 0.00200	Time 0.275 (0.620)	Data 0.000 (0.049)	Loss 2.0600 (2.8127)	Prec@1 35.156 (31.392)	Prec@5 90.625 (82.031)
Epoch: [0][20/97], lr: 0.00200	Time 0.279 (0.459)	Data 0.000 (0.033)	Loss 1.7687 (2.4285)	Prec@1 46.094 (38.802)	Prec@5 90.625 (86.570)
Epoch: [0][30/97], lr: 0.00200	Time 0.282 (0.402)	Data 0.000 (0.028)	Loss 1.5034 (2.2222)	Prec@1 51.562 (42.314)	Prec@5 92.188 (88.256)
Epoch: [0][40/97], lr: 0.00200	Time 0.294 (0.373)	Data 0.000 (0.025)	Loss 1.1810 (2.0436)	Prec@1 58.594 (44.512

In [1]:
import torchvision
import torch.nn as nn
import torch
import models
from torchsummary import summary

In [2]:
def off_diagonal(x):
    # return a flattened view of the off-diagonal elements of a square matrix
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

In [3]:
class BarlowTwins(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.backbone = torchvision.models.resnet50(zero_init_residual=True)
        self.backbone.fc = nn.Identity()

        # projector
        sizes = [2048] + [8192, 8192, 8192]
        layers = []
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=False))
            layers.append(nn.BatchNorm1d(sizes[i + 1]))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Linear(sizes[-2], sizes[-1], bias=False))
        self.projector = nn.Sequential(*layers)

        # normalization layer for the representations z1 and z2
        self.bn = nn.BatchNorm1d(sizes[-1], affine=False)

    def forward(self, y1, y2):
        z1 = self.projector(self.backbone(y1))
        z2 = self.projector(self.backbone(y2))

        # empirical cross-correlation matrix
        c = self.bn(z1).T @ self.bn(z2)

        return c

In [2]:
model = models.resnet32()

num_trans : 16


In [3]:
print(model)

ResNet_s(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affin